In [22]:
import pandas as pd
from collections import Counter
from pathlib import Path

# ==========================
# Cấu hình
# ==========================
TOP_N = 50  # ⬅️ chỉnh ở đây nếu muốn đổi số lượng mã lấy
DATA_ROOT = Path("../data")
HOSP_DIR = DATA_ROOT / "mimiciv" / "3.1" / "hosp"
PROC_DIR = DATA_ROOT / "mimiciv_subset"
PROC_DIR.mkdir(parents=True, exist_ok=True)

# ==========================
# HÀM: LẤY & LƯU TOP MÃ PHỔ BIẾN
# ==========================
def export_top_codes(HOSP_DIR, PROC_DIR, TOP_N=50):
    # --- 1️⃣ ICD (diagnoses_icd) ---
    diag_path = HOSP_DIR / "diagnoses_icd.csv.gz"
    print("📖 Đang xử lý ICD...")
    diag_cnt = Counter()
    for chunk in pd.read_csv(diag_path, usecols=["hadm_id", "icd_code"], compression="gzip", chunksize=200_000):
        chunk = chunk.dropna(subset=["hadm_id", "icd_code"])
        diag_cnt.update(chunk["icd_code"].astype(str))
    diag_df = pd.DataFrame(diag_cnt.items(), columns=["icd_code", "count"]).sort_values("count", ascending=False)
    diag_top = diag_df.head(TOP_N)
    out_icd = PROC_DIR / "icd.csv"
    diag_top.to_csv(out_icd, index=False)
    print(f"✅ Lưu {TOP_N} mã ICD → {out_icd} ({len(diag_df):,} mã, tổng {diag_df['count'].sum():,} lượt)")

    # --- 2️⃣ Procedures (procedures_icd) ---
    proc_path = HOSP_DIR / "procedures_icd.csv.gz"
    print("🛠️  Đang xử lý thủ thuật...")
    proc_cnt = Counter()
    for chunk in pd.read_csv(proc_path, usecols=["hadm_id", "icd_code"], compression="gzip", chunksize=200_000):
        chunk = chunk.dropna(subset=["hadm_id", "icd_code"])
        proc_cnt.update(chunk["icd_code"].astype(str))
    proc_df = pd.DataFrame(proc_cnt.items(), columns=["proc_code", "count"]).sort_values("count", ascending=False)
    proc_top = proc_df.head(TOP_N)
    out_proc = PROC_DIR / "procedures.csv"
    proc_top.to_csv(out_proc, index=False)
    print(f"✅ Lưu {TOP_N} thủ thuật → {out_proc} ({len(proc_df):,} mã, tổng {proc_df['count'].sum():,} lượt)")

    # --- 3️⃣ Labs (labevents + d_labitems) ---
    lab_path = HOSP_DIR / "labevents.csv.gz"
    dlab_path = HOSP_DIR / "d_labitems.csv.gz"
    print("🧪 Đang xử lý xét nghiệm...")
    lab_cnt = Counter()
    for chunk in pd.read_csv(lab_path, usecols=["hadm_id", "itemid"], compression="gzip", chunksize=200_000):
        chunk = chunk.dropna(subset=["hadm_id", "itemid"])
        lab_cnt.update(chunk["itemid"].astype(int))
    lab_df = pd.DataFrame(lab_cnt.items(), columns=["itemid", "count"]).sort_values("count", ascending=False)

    # Gắn label từ d_labitems
    dlab = pd.read_csv(dlab_path, usecols=["itemid", "label"], compression="gzip")
    lab_df = lab_df.merge(dlab, on="itemid", how="left")
    lab_top = lab_df.head(TOP_N)
    out_lab = PROC_DIR / "labs.csv"
    lab_top.to_csv(out_lab, index=False)
    print(f"✅ Lưu {TOP_N} xét nghiệm → {out_lab} ({len(lab_df):,} mã, tổng {lab_df['count'].sum():,} lượt)")

    # --- Tổng kết ---
    print("\n=== TỔNG KẾT ===")
    print(f"- Tổng mã ICD: {len(diag_df):,}")
    print(f"- Tổng mã thủ thuật: {len(proc_df):,}")
    print(f"- Tổng mã xét nghiệm: {len(lab_df):,}")
    print(f"- Tổng lượt ICD ghi nhận: {diag_df['count'].sum():,}")
    print(f"- Tổng lượt thủ thuật ghi nhận: {proc_df['count'].sum():,}")
    print(f"- Tổng lượt xét nghiệm ghi nhận: {lab_df['count'].sum():,}")
    print(f"📂 Các file đã lưu trong {PROC_DIR.resolve()}")

# ==========================
# GỌI HÀM
# ==========================
export_top_codes(HOSP_DIR, PROC_DIR, TOP_N)


📖 Đang xử lý ICD...
✅ Lưu 50 mã ICD → ../data/mimiciv_subset/icd.csv (28,562 mã, tổng 6,364,488 lượt)
🛠️  Đang xử lý thủ thuật...
✅ Lưu 50 thủ thuật → ../data/mimiciv_subset/procedures.csv (14,911 mã, tổng 859,655 lượt)
🧪 Đang xử lý xét nghiệm...
✅ Lưu 50 xét nghiệm → ../data/mimiciv_subset/labs.csv (896 mã, tổng 84,605,867 lượt)

=== TỔNG KẾT ===
- Tổng mã ICD: 28,562
- Tổng mã thủ thuật: 14,911
- Tổng mã xét nghiệm: 896
- Tổng lượt ICD ghi nhận: 6,364,488
- Tổng lượt thủ thuật ghi nhận: 859,655
- Tổng lượt xét nghiệm ghi nhận: 84,605,867
📂 Các file đã lưu trong /Users/lehoangkhang/Tài liệu/revita-sympdiag/data/mimiciv_subset


In [23]:
import pandas as pd
from pathlib import Path

# ==========================
# Cấu hình đường dẫn
# ==========================
DATA_ROOT = Path("../data")
HOSP_DIR = DATA_ROOT / "mimiciv" / "3.1" / "hosp"
NOTE_DIR = DATA_ROOT / "mimic-iv-note" / "2.2" / "note"
SUBSET_DIR = DATA_ROOT / "mimiciv_subset"
SUBSET_DIR.mkdir(parents=True, exist_ok=True)

PROC_DIR = SUBSET_DIR

# ==========================
# 1️⃣ Đọc top 50 mã phổ biến
# ==========================
top_icd = pd.read_csv(PROC_DIR / "icd.csv")["icd_code"].astype(str).tolist()
top_proc = pd.read_csv(PROC_DIR / "procedures.csv")["proc_code"].astype(str).tolist()
top_lab = pd.read_csv(PROC_DIR / "labs.csv")["itemid"].astype(int).tolist()

print(f"🔍 Đọc vocab top50: ICD={len(top_icd)}, PROC={len(top_proc)}, LAB={len(top_lab)}")

# ==========================
# 2️⃣ Hàm tiện ích: lấy danh sách hadm_id theo mã
# ==========================
def get_hadm_with_codes(path, col_code, top_list, dtype="str"):
    hadm_set = set()
    for chunk in pd.read_csv(path, usecols=["hadm_id", col_code], chunksize=200_000):
        chunk = chunk.dropna(subset=["hadm_id", col_code])
        if dtype == "str":
            chunk[col_code] = chunk[col_code].astype(str)
        else:
            chunk[col_code] = chunk[col_code].astype(int)
        matched = chunk.loc[chunk[col_code].isin(top_list), "hadm_id"].astype(int)
        hadm_set.update(matched.tolist())
    return hadm_set

# ==========================
# 3️⃣ Lọc theo từng loại
# ==========================
print("📖 Đang lọc hadm theo top ICD...")
hadm_icd = get_hadm_with_codes(HOSP_DIR / "diagnoses_icd.csv.gz", "icd_code", top_icd, "str")
print(f"→ {len(hadm_icd):,} lượt nhập viện có ICD trong top 50")

print("🛠️  Đang lọc hadm theo top thủ thuật...")
hadm_proc = get_hadm_with_codes(HOSP_DIR / "procedures_icd.csv.gz", "icd_code", top_proc, "str")
print(f"→ {len(hadm_proc):,} lượt nhập viện có thủ thuật trong top 50")

print("🧪 Đang lọc hadm theo top xét nghiệm...")
hadm_lab = get_hadm_with_codes(HOSP_DIR / "labevents.csv.gz", "itemid", top_lab, "int")
print(f"→ {len(hadm_lab):,} lượt nhập viện có xét nghiệm trong top 50")

print("🩺 Đang lọc hadm có ghi chú xuất viện...")
hadm_note = set()
for chunk in pd.read_csv(NOTE_DIR / "discharge.csv.gz", usecols=["hadm_id", "text"], chunksize=200_000):
    chunk = chunk.dropna(subset=["hadm_id", "text"])
    hadm_note.update(chunk["hadm_id"].astype(int).tolist())
print(f"→ {len(hadm_note):,} lượt nhập viện có note xuất viện")

# ==========================
# 4️⃣ Giao 4 tập
# ==========================
hadm_all = hadm_icd & hadm_proc & hadm_lab & hadm_note
print(f"\n✅ Tổng số lượt nhập viện có đủ 4 yếu tố (ICD + PROC + LAB + NOTE): {len(hadm_all):,}")

# ==========================
# 5️⃣ Lọc bảng admissions và lưu ra CSV thường
# ==========================
adm_path = HOSP_DIR / "admissions.csv.gz"
print("📘 Đang lọc bảng admissions...")

admissions = pd.read_csv(
    adm_path,
    usecols=["hadm_id", "subject_id", "admittime", "dischtime", "deathtime"],
    compression="gzip",
    parse_dates=["admittime", "dischtime", "deathtime"],
)

admissions = admissions[admissions["hadm_id"].isin(hadm_all)].copy()
admissions = admissions.sort_values("admittime")

out_path = SUBSET_DIR / "admissions.csv"
admissions.to_csv(out_path, index=False)  # 💾 không nén gzip

print(f"📂 Đã lưu subset admissions với {len(admissions):,} lượt nhập viện → {out_path.resolve()}")
print("Bao gồm các cột: hadm_id, subject_id, admittime, dischtime, deathtime")


🔍 Đọc vocab top50: ICD=50, PROC=50, LAB=50
📖 Đang lọc hadm theo top ICD...
→ 454,431 lượt nhập viện có ICD trong top 50
🛠️  Đang lọc hadm theo top thủ thuật...
→ 145,381 lượt nhập viện có thủ thuật trong top 50
🧪 Đang lọc hadm theo top xét nghiệm...
→ 441,727 lượt nhập viện có xét nghiệm trong top 50
🩺 Đang lọc hadm có ghi chú xuất viện...
→ 331,793 lượt nhập viện có note xuất viện

✅ Tổng số lượt nhập viện có đủ 4 yếu tố (ICD + PROC + LAB + NOTE): 87,951
📘 Đang lọc bảng admissions...
📂 Đã lưu subset admissions với 87,951 lượt nhập viện → /Users/lehoangkhang/Tài liệu/revita-sympdiag/data/mimiciv_subset/admissions.csv
Bao gồm các cột: hadm_id, subject_id, admittime, dischtime, deathtime


In [24]:
import pandas as pd
from pathlib import Path

# ==========================
# Cấu hình
# ==========================
DATA_ROOT = Path("../data")
HOSP_DIR = DATA_ROOT / "mimiciv" / "3.1" / "hosp"
SUBSET_DIR = DATA_ROOT / "mimiciv_subset"
SUBSET_DIR.mkdir(parents=True, exist_ok=True)

# ==========================
# 1️⃣ Đọc danh sách bệnh nhân trong subset admissions
# ==========================
adm_subset_path = SUBSET_DIR / "admissions.csv"
if not adm_subset_path.exists():
    adm_subset_path = SUBSET_DIR / "admissions.csv.gz"

admissions = pd.read_csv(adm_subset_path, usecols=["hadm_id", "subject_id"])
subject_ids = set(admissions["subject_id"].dropna().astype(int).tolist())

print(f"🔍 Tổng số bệnh nhân cần giữ: {len(subject_ids):,}")

# ==========================
# 2️⃣ Đọc và lọc bảng patients
# ==========================
patients_path = HOSP_DIR / "patients.csv.gz"
usecols = ["subject_id", "gender", "anchor_age", "anchor_year"]

patients = pd.read_csv(patients_path, usecols=usecols, compression="gzip")
patients = patients[patients["subject_id"].isin(subject_ids)].copy()

print(f"✅ Đã lọc {len(patients):,} bệnh nhân trong tập admissions subset")

# ==========================
# 3️⃣ Chuẩn hoá giới tính và lưu CSV (không nén)
# ==========================
patients["gender"] = patients["gender"].astype(str).str.upper().str[0]  # chuẩn hóa 'M', 'F', 'U'

out_path = SUBSET_DIR / "patients.csv"
patients.to_csv(out_path, index=False)

print(f"📂 Đã lưu file rút gọn → {out_path.resolve()}")
print("Bao gồm các cột: subject_id, gender, anchor_age, anchor_year")


🔍 Tổng số bệnh nhân cần giữ: 52,610
✅ Đã lọc 52,610 bệnh nhân trong tập admissions subset
📂 Đã lưu file rút gọn → /Users/lehoangkhang/Tài liệu/revita-sympdiag/data/mimiciv_subset/patients.csv
Bao gồm các cột: subject_id, gender, anchor_age, anchor_year


In [25]:
import pandas as pd
from pathlib import Path
import re

# ==========================
# Cấu hình đường dẫn
# ==========================
DATA_ROOT = Path("../data")
NOTE_DIR = DATA_ROOT / "mimic-iv-note" / "2.2" / "note"
SUBSET_DIR = DATA_ROOT / "mimiciv_subset"
SUBSET_DIR.mkdir(parents=True, exist_ok=True)

# ==========================
# Đọc danh sách hadm_id subset
# ==========================
admissions = pd.read_csv(SUBSET_DIR / "admissions.csv", usecols=["hadm_id"])
hadm_ids = set(admissions["hadm_id"].astype(int).tolist())
print(f"🔍 Tổng số lượt nhập viện cần giữ: {len(hadm_ids):,}")

# ==========================
# Hàm làm sạch ghi chú (cắt đến *trước* 'Service:')
# ==========================
def clean_discharge_text(text: str) -> str:
    if pd.isna(text):
        return ""
    s = str(text)

    # 🚩 Tìm "Service:" và cắt bỏ mọi thứ trước nó (nhưng vẫn giữ 'Service:')
    idx = s.find("Service:")
    if idx != -1:
        s = s[idx:]  # giữ nguyên từ 'Service:' trở đi

    # Làm sạch ký tự đặc biệt / dòng trang trí
    s = re.sub(r"[-_=]{3,}", " ", s)
    s = re.sub(r"\s+", " ", s).strip()

    # Giới hạn độ dài để tránh file quá lớn
    return s[:4000]

# ==========================
# Đọc & lọc discharge note
# ==========================
discharge_path = NOTE_DIR / "discharge.csv.gz"
rows = []

print("🩺 Đang đọc và làm sạch ghi chú xuất viện...")
for chunk in pd.read_csv(discharge_path, usecols=["hadm_id", "charttime", "text"], compression="gzip", chunksize=200_000):
    chunk = chunk.dropna(subset=["hadm_id", "text"]).copy()
    chunk["hadm_id"] = chunk["hadm_id"].astype(int)
    chunk = chunk[chunk["hadm_id"].isin(hadm_ids)]
    if chunk.empty:
        continue
    chunk["charttime"] = pd.to_datetime(chunk["charttime"], errors="coerce")
    chunk["text"] = chunk["text"].map(clean_discharge_text)
    rows.append(chunk[["hadm_id", "charttime", "text"]])

# ==========================
# Gộp & chọn note mới nhất
# ==========================
if rows:
    discharge_df = pd.concat(rows, ignore_index=True)
    discharge_df = (
        discharge_df.sort_values(["hadm_id", "charttime"], ascending=[True, False])
        .drop_duplicates(subset=["hadm_id"], keep="first")
    )
else:
    discharge_df = pd.DataFrame(columns=["hadm_id", "charttime", "text"])

print(f"✅ Đã lọc & làm sạch {len(discharge_df):,} lượt nhập viện có ghi chú xuất viện")

# ==========================
# Lưu CSV
# ==========================
out_path = SUBSET_DIR / "discharge.csv"
discharge_df.to_csv(out_path, index=False)
print(f"📂 Đã lưu file ghi chú xuất viện → {out_path.resolve()}")
print("Bao gồm các cột: hadm_id, charttime, text (đã bỏ phần đầu nhưng vẫn giữ 'Service:').")


🔍 Tổng số lượt nhập viện cần giữ: 87,951
🩺 Đang đọc và làm sạch ghi chú xuất viện...
✅ Đã lọc & làm sạch 87,951 lượt nhập viện có ghi chú xuất viện
📂 Đã lưu file ghi chú xuất viện → /Users/lehoangkhang/Tài liệu/revita-sympdiag/data/mimiciv_subset/discharge.csv
Bao gồm các cột: hadm_id, charttime, text (đã bỏ phần đầu nhưng vẫn giữ 'Service:').


In [27]:
import pandas as pd
from pathlib import Path

# ==========================
# Cấu hình đường dẫn
# ==========================
DATA_ROOT = Path("../data")
HOSP_DIR = DATA_ROOT / "mimiciv" / "3.1" / "hosp"
SUBSET_DIR = DATA_ROOT / "mimiciv_subset"
SUBSET_DIR.mkdir(parents=True, exist_ok=True)

# ==========================
# 1️⃣ Hàm tiện ích
# ==========================
def export_filtered_dict(top_file, dict_file, out_name, code_col="icd_code"):
    print(f"\n📘 Xử lý {top_file.name} → {out_name}")
    
    # Đọc danh sách top
    top_df = pd.read_csv(top_file)
    top_codes = top_df[top_df.columns[0]].astype(str).tolist()
    
    # Đọc dictionary gốc
    dict_df = pd.read_csv(dict_file, compression="gzip")
    dict_df[code_col] = dict_df[code_col].astype(str)
    
    # Lọc theo top
    filtered = dict_df[dict_df[code_col].isin(top_codes)].copy()
    
    # Merge thêm cột count để tiện tra cứu
    filtered = filtered.merge(top_df, left_on=code_col, right_on=top_df.columns[0], how="left")
    filtered = filtered.sort_values("count", ascending=False)
    
    # Lưu file với tên gốc
    out_path = SUBSET_DIR / out_name
    filtered.to_csv(out_path, index=False)
    
    print(f"✅ Lưu {len(filtered):,} dòng → {out_path}")
    return filtered

# ==========================
# 2️⃣ ICD Diagnoses
# ==========================
diag_filtered = export_filtered_dict(
    top_file = SUBSET_DIR / "icd.csv",
    dict_file = HOSP_DIR / "d_icd_diagnoses.csv.gz",
    out_name  = "d_icd_diagnoses.csv",
    code_col  = "icd_code"
)

# ==========================
# 3️⃣ ICD Procedures
# ==========================
proc_filtered = export_filtered_dict(
    top_file = SUBSET_DIR / "procedures.csv",
    dict_file = HOSP_DIR / "d_icd_procedures.csv.gz",
    out_name  = "d_icd_procedures.csv",
    code_col  = "icd_code"
)

# ==========================
# 4️⃣ Tổng kết
# ==========================
print("\n=== TỔNG KẾT ===")
print(f"- d_icd_diagnoses.csv: {len(diag_filtered):,} dòng")
print(f"- d_icd_procedures.csv: {len(proc_filtered):,} dòng")
print(f"📂 Đã lưu vào: {SUBSET_DIR.resolve()}")



📘 Xử lý icd.csv → d_icd_diagnoses.csv
✅ Lưu 50 dòng → ../data/mimiciv_subset/d_icd_diagnoses.csv

📘 Xử lý procedures.csv → d_icd_procedures.csv
✅ Lưu 50 dòng → ../data/mimiciv_subset/d_icd_procedures.csv

=== TỔNG KẾT ===
- d_icd_diagnoses.csv: 50 dòng
- d_icd_procedures.csv: 50 dòng
📂 Đã lưu vào: /Users/lehoangkhang/Tài liệu/revita-sympdiag/data/mimiciv_subset
